In [ ]:
!pip install -q gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import openai
import gradio as gr

# Set your OpenAI API key
openai.api_key = "your_key"

# Function to generate a refined image prompt
def generate_and_refine_prompt(
    quiz_name,  # Internal metadata, not included in the response
    level,      # Beginner, Intermediate, Advanced
    theme,      # e.g., Nature, Fantasy
    mood,       # e.g., Joyful, Peaceful
    characters, # e.g., Animals, Humans
    context,    # e.g., Indoor, Outdoor
    art_medium, # e.g., Watercolors, 3D
    color_scheme, # e.g., Vibrant, Dark
    shot_type,  # e.g., Close-up, Wide-angle
    actions_details, # Additional details
    negative_prompt # Things to avoid
):
    """
    Generate a creative and structured image prompt based on user-selected options.
    """

    # Step 1: Define the base instruction for the AI
    if "Beginner" in level:
        complexity_instruction = "The lenght cannot be more than 50 tokens.Create a simple and short prompt with minimal details."
    elif "Intermediate" in level:
        complexity_instruction = "The lenght cannot be more than 77 tokens.Create a moderately descriptive prompt with some complexity by combining multiple ideas like style, objects, and mood."
    elif "Advanced" in level:
        complexity_instruction = "The lenght cannot be more than 150 tokens .Create a highly detailed and professional-quality prompt. Include nuanced details.Should be written in one passage."

    ai_instruction = (
        f"You are a professional prompt engineer specializing in creating creative, detailed, and structured prompts "
        f"for AI image generation tool Stable diffusion 3.5. Your job is to use the provided keywords and generate an example "
        f"of a well-crafted image-generation prompt. Adjust the complexity based on the following instruction: {complexity_instruction}."
    )

    # Step 2: Combine user inputs into keywords
    keywords = ""
    if theme:
        keywords += f"Theme: {', '.join(theme)}. "
    if mood:
        keywords += f"Mood: {', '.join(mood)}. "
    if characters:
        keywords += f"Characters: {', '.join(characters)}. "
    if context:
        keywords += f"Context: {', '.join(context)}. "
    if art_medium:
        keywords += f"Art Medium: {', '.join(art_medium)}. "
    if color_scheme:
        keywords += f"Color Scheme: {', '.join(color_scheme)}. "
    if shot_type:
        keywords += f"Shot Type: {', '.join(shot_type)}. "
    if actions_details:
        keywords += f"Details: {actions_details}. "
    if negative_prompt:
        keywords += f"Exclude: {negative_prompt}. "

    # Step 3: Create the full input for GPT
    full_prompt = f"{ai_instruction}\n\nKeywords:\n{keywords}\n\nGenerate an example of a stable diffusion prompt based on these keywords."

    # Step 4: Call the OpenAI API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Use GPT-4 for best results
            messages=[
                {"role": "system", "content": "You are a professional stable diffusion prompt engineer."},
                {"role": "user", "content": full_prompt},
            ],
            max_tokens=200,
            temperature=0.7,
        )
        # Extract and return the generated prompt
        generated_prompt = response['choices'][0]['message']['content']
        return generated_prompt

    except Exception as e:
        return f"An error occurred while refining the prompt: {e}"

# Define the Gradio interface
interface = gr.Interface(
    fn=generate_and_refine_prompt,
    inputs=[
        gr.Textbox(label="Quiz Name", placeholder="Optional (not sent to AI)"),
        gr.CheckboxGroup(["Beginner", "Intermediate", "Advanced"], label="Choose Level"),
        gr.CheckboxGroup(
            ["Nature", "Storytelling", "Fantasy and Magic", "Historical", "Everyday Life"],
            label="Choose Theme",
        ),
        gr.CheckboxGroup(
            ["Joyful", "Energetic", "Hope", "Peaceful", "Cosy"], label="Choose a Mood"
        ),
        gr.CheckboxGroup(
            ["Animals", "Humans", "Fantasy Characters", "None"], label="Choose Characters"
        ),
        gr.CheckboxGroup(["Indoor", "Outdoor", "At Night", "At Morning"], label="Choose Context"),
        gr.CheckboxGroup(
            ["Pencil Drawing", "Graffiti", "Water Colors", "3D"], label="Choose Art Medium"
        ),
        gr.CheckboxGroup(["Vibrant", "Dark", "Pastel"], label="Choose Color Scheme"),
        gr.CheckboxGroup(["Close-up", "Wide-angle", "Long-shot"], label="Choose Shot Type"),
        gr.Textbox(label="Enter Actions or Details (Optional)", placeholder="e.g., Wearing a suit"),
        gr.Textbox(
            label="Enter Negative Prompt (Optional)",
            placeholder="e.g., Blurry image, Mutated body parts",
        ),
    ],
    outputs="textbox",
    title="AI-Generated Image Prompt Tool",
    description="Select options to generate an example of a structured and creative image-generation prompt.",
)

# Launch the Gradio app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ec463d4213ea86b01.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
